In [26]:
from langchain_community.document_loaders import PyPDFLoader


In [27]:
file_path = "data/414759-1-_5_Nike-NPS-Combo_Form-10-K_WR.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

106


In [28]:
print(docs[0].page_content[0:100])
print(docs[0].metadata)

FORM 10-K FORM 10-K
{'source': 'data/414759-1-_5_Nike-NPS-Combo_Form-10-K_WR.pdf', 'page': 0}


In [33]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer


In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [35]:
llm_model = AutoModelForCausalLM.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-3B-v1", torch_dtype=torch.float16)
llm_model = llm_model.to(device)

embeddings_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings_model = embeddings_model.to(device)

d:\__repos\ZZSN24L\venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [38]:
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document

# Initialize the SentenceTransformer model
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Create a function to get embeddings
def get_embeddings(documents):
    # texts = [doc.text for doc in documents]
    texts = [doc.page_content for doc in documents]
    return embeddings_model.encode(texts)

# Split the documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(loader.load())



In [40]:

# Get embeddings for the document splits
embeddings = get_embeddings(splits)

# Initialize Chroma vector store with the documents and their embeddings
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
# vectorstore = Chroma.from_documents(documents=splits, embedding_function=embeddings_model)

# Get the retriever
retriever = vectorstore.as_retriever()

AttributeError: 'numpy.ndarray' object has no attribute 'embed_documents'

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "What was Nike's revenue in 2023?"})

results

In [ ]:
print(results["context"][0].page_content)

In [ ]:
print(results["context"][0].metadata)